# Ruteo

## Preparación de la Sesión

In [1]:
!git clone https://github.com/JuanCruzC97/ope3-logistica.git &> /dev/null

In [6]:
cd ope3-logistica

/content/ope3-logistica


In [7]:
!pip install -e . &> /dev/null

## Módulos

In [9]:
import pandas as pd
import numpy as np
import plotly.express as px

In [10]:
from logistica.ruteo import Ruteo
from logistica.utils import load_inputs, preparar_df_pedidos

In [11]:
from logistica import metaheuristicas as mh

# Carga de Datos

In [12]:
df_clientes = load_inputs("clientes")
df_pedidos = load_inputs("pedidos")
df_pedidos = pd.merge(left=df_pedidos, right=df_clientes, on="cliente", how="inner")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [13]:
df_camiones = load_inputs("camiones")
df_camiones

,camion,carga_max,pedidos_max,dist_max
0,1,12,3,2
1,2,12,3,2
2,3,12,3,2
3,4,12,3,2
4,5,12,3,2
5,6,12,3,2


# Optimización 1 Pedido


In [102]:
#@title Seleccionar Pedido
pedido = 'pedidos6' #@param ["pedidos1", "pedidos2", "pedidos3", "pedidos4", "pedidos5", "pedidos6"]

In [103]:
df_pedido = preparar_df_pedidos(df_pedidos, pedido)
df_pedido

,cliente,pedidos,coord_x,coord_y
0,A,6,0.257571,1.803726
1,B,3,1.523313,2.102301
2,C,7,0.710910,2.629800
3,D,2,1.013940,2.149740
4,E,3,0.609480,0.666810
5,F,5,0.998640,1.451790
6,G,3,1.434510,0.018180
7,H,8,2.171880,0.764910
8,I,6,1.412280,0.863190
9,J,2,2.386260,1.387260


In [104]:
#@title Parámetros del Ruteo
costo_oportunidad = 3000 #@param {type:"number"}
presupuesto = 1220 #@param {type:"number"}

In [105]:
ruteo = Ruteo(df_camiones, df_pedido, costo_oportunidad=costo_oportunidad, presupuesto=presupuesto)

In [106]:
#@title Parámetros 
mode = 'random' #@param ["simple", "random"]
random_state = 45 #@param {type:"number"}

In [109]:
ruteo.get_solucion_inicial(mode=mode, random_state=random_state)
ruteo

--Ruteo--
Carga Total: 54tn
Costo Camiones: 59000$
Costo Oportunidad: 21000$ 
Costo Total: 80000$ 
Costo Total por tn: 1481.48$/tn 
Ahorro: 21.43%

In [110]:
#@title Parámetros Metaheurística
temp_inicial = 100 #@param {type:"number"}
temp_final = 1 #@param {type:"number"}
k = 200 #@param {type:"number"}
iters = 20 #@param {type:"number"}
temp_mode = 'non_linear' #@param ["linear", "non_linear"]
p = 1 #@param {type:"number"}
random_state = 42 #@param {type:"number"}

In [112]:
best_sol, history = mh.sa(ruteo, 
                          t_inicial=temp_inicial, 
                          t_final=temp_final, 
                          k=k, 
                          iters=iters, 
                          temp_mode=temp_mode, 
                          prob=p, 
                          random_state=random_state)
best_sol

100%|██████████| 200/200 [00:04<00:00, 47.57it/s]


--Ruteo--
Carga Total: 61tn
Costo Camiones: 61000$
Costo Oportunidad: 0$ 
Costo Total: 61000$ 
Costo Total por tn: 1000.0$/tn 
Ahorro: -18.03%

In [115]:
best_sol.summary_camiones()

,Cliente 1,Cliente 2,Cliente 3,Costo,Carga,Costo_tn,Ahorro
Camion 1,F,O,N,10000,10,1000.0,-18.03
Camion 2,B,A,M,10000,10,1000.0,-18.03
Camion 3,K,L,None,10000,10,1000.0,-18.03
Camion 4,E,C,None,10000,10,1000.0,-18.03
Camion 5,I,J,G,11000,11,1000.0,-18.03
Camion 6,H,D,None,10000,10,1000.0,-18.03
Costo Oportunidad,,,,0,0,,
Total,,,,61000,61,1000.0,-18.03


In [114]:
best_sol.summary_pedidos()

,Carga,Asignado,Camion
Pedido A,6,True,2
Pedido B,3,True,2
Pedido C,7,True,4
Pedido D,2,True,6
Pedido E,3,True,4
Pedido F,5,True,1
Pedido G,3,True,5
Pedido H,8,True,6
Pedido I,6,True,5
Pedido J,2,True,5


In [116]:
df_history = pd.DataFrame({"temp":history["temp"],
                           "actual_sol":history["actual_sol"],
                           "new_sol":history["new_sol"]})

df_history = (df_history
              .assign(delta = lambda df_: df_.actual_sol - df_.new_sol,
                      p = lambda df_: round(np.exp(df_.delta/df_.temp), 2)))

df_history

,temp,actual_sol,new_sol,delta,p
0,99.849882,1481.48,1481.48,0.00,1.00
1,99.849882,1481.48,1547.17,-65.69,0.52
2,99.849882,1547.17,1547.17,0.00,1.00
3,99.849882,1547.17,1547.17,0.00,1.00
4,99.849882,1547.17,1535.85,11.32,1.12
...,...,...,...,...,...
3995,0.091105,1000.00,1000.00,0.00,1.00
3996,0.091105,1000.00,1026.23,-26.23,0.00
3997,0.091105,1000.00,1000.00,0.00,1.00
3998,0.091105,1000.00,1029.51,-29.51,0.00


In [117]:
px.line(df_history,
        x=df_history.index,
        y=["new_sol", "actual_sol"],
        width=800,
        template="plotly_white").show()

px.line(df_history,
        x=df_history.index,
        y=["delta", "p"],
        width=800,
        template="plotly_white").show()

px.line(df_history,
        x=df_history.index,
        y=["temp"],
        width=800,
        template="plotly_white").show()

px.line(df_history,
        x=df_history.index,
        y=df_history.actual_sol.cummin(),
        width=800,
        template="plotly_white").show()